In [20]:
##image taking function
import cv2

def takeImage():
    check, frame = cam.read()
    cv2.imwrite(r"test.png", frame)

In [21]:
##NN

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

import os

# Function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath, map_location=torch.device('cpu')) ###ACHTUNG! auf CPU...
    
    if checkpoint['arch'] == 'vgg16':
        
        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 7000)),
                                            ('relu', nn.ReLU()),
                                            ('drop', nn.Dropout(p=0.5)),
                                            ('fc2', nn.Linear(7000, 21)),
                                            ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    return model
## problems opening via relative path -> https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python
script_dir = os.getcwd() #os.path.dirname(__file__) #<-- absolute dir the script is in (workingdir)
rel_path = 'NNs\\ABCDEFGHIJKLMNOPQRdns4Gextra.pth'
abs_file_path = os.path.join(script_dir, rel_path)

model = load_checkpoint(abs_file_path)
# model.cuda() ##STAY ON CPU
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
import json

## problems opening via relative path -> https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python
script_dir = os.getcwd() #os.path.dirname(__file__) #<-- absolute dir the script is in (workingdir)
rel_path = 'jsons\\big.json'
abs_file_path = os.path.join(script_dir, rel_path)

with open(abs_file_path, 'r') as f:
    number_to_ASL = json.load(f)

print(number_to_ASL)    
    

{'1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E', '6': 'F', '7': 'G', '8': 'H', '9': 'I', '10': 'J', '11': 'K', '12': 'L', '13': 'M', '14': 'N', '15': 'O', '16': 'P', '17': 'Q', '18': 'R', '19': 'S', '20': 'T', '21': 'U', '22': 'V', '23': 'W', '24': 'X', '25': 'Y', '26': 'Z', '27': 'del', '28': 'nothing', '29': 'space'}


In [23]:
from PIL import Image

def process_image(image_path):
    # makes numpy image
    
    pil_image = Image.open(image_path)
    
    # Resize
    if pil_image.size[0] > pil_image.size[1]:
        pil_image.thumbnail((5000, 256))
    else:
        pil_image.thumbnail((256, 5000))
        
    # Crop 
    left_margin = (pil_image.width-224)/2
    bottom_margin = (pil_image.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    
    pil_image = pil_image.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    pil_image.save('cropped.png')
    
    # Normalize
    np_image = np.array(pil_image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std
    

    np_image = np_image.transpose((2, 0, 1))
    
    return np_image

def predict(image_path, model, topk=3):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    image = process_image(image_path)
    
    # Convert image to PyTorch tensor first
    image = torch.from_numpy(image).type(torch.FloatTensor)
    
    # Returns a new tensor with a dimension of size one inserted at the specified position.
    image = image.unsqueeze(0)
    
    output = model.forward(image)
    
    probabilities = torch.exp(output)
    
    # Probabilities and the indices of those probabilities corresponding to the classes
    top_probabilities, top_indices = probabilities.topk(topk)
    
    # Convert to lists
    top_probabilities = top_probabilities.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    top_indices = top_indices.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    
    # Convert topk_indices to the actual class labels using class_to_idx
    # Invert the dictionary so you get a mapping from index to class.
    
    idx_to_class = {value: key for key, value in model.class_to_idx.items()}
    #print(idx_to_class)
    
    top_classes = [idx_to_class[index] for index in top_indices]
    
    return top_probabilities, top_classes


# pre-ideas:
main loop analyzes *sequence* of predictions (over time) -> advantage of comparision, not the first best suggestion is recognized; goal is to 'flatten' _*somehow* (with functions)_ the noise, small fluctuations of the prediction propabilities;

#### Different approaches: 
1. if sequence has all the time **one** leader, take it
* look, if the last **two** (or top n) highest ranked classes*(if no big prop difference)* were amongst the sequence
2. use means
2. sum the probs of the sequence; take the "highest"   ⋅⋅⋅⋅⋅⋅⋅⋅⋅
    *if difference is small, consider if you really make a decision or if you wait*
2. arithmetic mean -> sum existence; weight in regard to rank position (1., 2., or 3. most likely prediction)⋅⋅⋅
    *if difference is small, consider if you really make a decision or if you wait*
    possibly improvable by considering propability differences (-> how telling the "rank" is)

#### Discussion:
1. very hardly compatible to slight variations / oscillations of the predictions due to little changes / insecureties
2. little broader casting than 1., but also not immune to pred oscillations
3. 
4. danger of honoring past events too much
5. " 

#### Conclusion:

1. and 
2. : Combine with other methods ; making hist short makes the whole systems sensitive for noise
3. 
4. and
5. Use gradients / look for pro-changes, not stable variables, which were the top choice some time ago even though new signs are with high precision recognized by the system  => gradients   **OR**    make past events count less using prefactors etc.

**For all:** 
Hardly any word has a 'Triple-B' or spells like 'NNNNNIIICCCEEEEE' => Uniform and cancel double chars out; *watch out, outstanding chars may disturb the spelling: 'BBBBAAABAAUUUUUMMMM' and can easily be detected...*
 
For testing purposes it would be nice to create an visualisation of the probs for all chars.
Or if multiple ways are realized, to have a way of comparision.

In [24]:
import seaborn as sns

def visualize_list(probs_list, name = 'dataplot.png'):
    #Visualisation
    global number_to_ASL
    ABC = [value for key, value in number_to_ASL.items()]
    ABC[-1] = "' '"
    ABC[-2] = "''"
    ABC[-3] = "dl"
    plt.figure()
    plt.bar(ABC, probs_list)
    plt.ylabel('Summed propabilities over hist with prefactors')
    plt.xlabel('Classes / Chars')
    plt.title(name)
    #Save to use it with pygame (is there an more intelligent solution?)
    plt.savefig(name)
    #plt.show
    plt.close()

def visualize_time_list(array, name = 'dataplot.png'):
    global number_to_ASL
    ABC = [value for key, value in number_to_ASL.items()]
    ABC[-1] = "' '"
    ABC[-2] = "''"
    ABC[-3] = "dl"
    ax = sns.heatmap(array, linewidth = 0.1)  # now using seaborn's heatmap
    plt.xticks(np.arange(29), ABC)
    plt.ylabel('Time of Prediction')
    plt.xlabel('Classes / Chars')
    plt.title(name)
    plt.savefig(name)
    plt.close()
    

    

In [25]:
import pygame  
pygame.init()     ##the pygame window
pygame.font.init()

#critical for algorithm
#Function to detect the actual sentence

# Casting - Klasse !
class DetectSentence:
    myfont = pygame.font.SysFont('Comic Sans MS', 30) ##For changes: Copied to the DetectSentence - Class

    def __init__(self, print_name, casting_function, sent_pos, dataplot_name = None, plot_pos = None):
        self.sentence = print_name
        self.readable_sentence = 'Casting N°'+print_name
        self.casting_hist = []
        self.dataplot_name = dataplot_name
        self.sent_pos = sent_pos
        self.plot_pos = plot_pos
        self.print_name = print_name
        self.last_char_nothing = False
        #Set 'casting_function'
        if casting_function == '2':
            self.casting_function = self.casting2
        elif casting_function == '4':
            self.casting_function = self.casting4
        elif casting_function == '5':
            self.casting_function = self.casting5
        elif casting_function == '6':
            self.casting_function = self.casting6
        elif casting_function == '7':
            self.casting_function = self.casting7
        else:
            print(casting_function + " failed")

        
    def draw_on_pygame(self, screen):

        sentsurface = self.myfont.render(self.readable_sentence, False, (0,0,0))
        screen.blit(sentsurface, self.sent_pos)
        
        try: #maybe not useful (because old img is loaded)
            plot_image = pygame.image.load(self.dataplot_name)
            screen.blit(plot_image, self.plot_pos)
        except:
            pass #print("Error occured plot image: " + self.print_name)

    def refresh_sentence(self, hist):    
        '''
        Two steps:
        1. search and find considerable / appropiate predictions for an char
        2. if the function deciders there is an plausible prediction, it expands the handed over sentence
        '''    
        global number_to_ASL

        best_choice_number = self.casting_function(hist)
        best_char = number_to_ASL[best_choice_number]

        if best_char == "nothing":
            self.last_char_nothing = True
            return
        elif best_char == "del":
            self.sentence = self.sentence[:-1]
        elif best_char == "space":
            self.sentence = self.sentence + ' '
        else:
            self.sentence = self.sentence + best_char
        
        #consistency_len =  1# could be any number, the longer the more strictly
        if self.sentence[-3] == self.sentence[-2] == self.sentence[-1]:
            if self.readable_sentence[-1] != self.sentence[-1]:
                self.readable_sentence += self.sentence[-1]
                self.last_char_nothing = False
            elif self.last_char_nothing:
                self.readable_sentence += self.sentence[-1]
                self.last_char_nothing = False

    
    ## 2. : look, if the last **two / (topn)** highest ranked classes*(if no big prop difference)* were amongst the sequence
    def casting2(self, hist):    
        topn = 3 # must be smaller than topk
        plausible_classes = list() #returns this list

        last_pred = hist[-1]

        for i in np.arange(topn):
            topn_pred_i = last_pred[i][0]

            plausible = True
            for hist_pred in hist:
                i_in_hist_pred = False
                for hist_pred_class in hist_pred:
                    if topn_pred_i == hist_pred_class[0]:
                        i_in_hist_pred = True
                if not i_in_hist_pred:
                    plausible = False

            if plausible:
                plausible_classes.append(topn_pred_i)
           
        #norm result to one answer string
        if plausible_classes:
            return plausible_classes[0]
        else:
            return '28'

    ## 4. sum the probs of the sequence; take the "highest"    *if difference is small, consider if you really make a decision or if you wait*
    def casting4(self, hist, dataplot_name = 'casting4.png'):
        probs_sums = np.zeros(29) # 30 or 29, but keep in mind:  change every i -> i-1

        for time_stamp in np.arange(len(hist)):    #possible prefactor timeline
            hist_pred = hist[time_stamp]
            for rank in np.arange(len(hist_pred)): #possible prefactor prediction rank
                hist_pred_class = hist_pred[rank]
                class_number = int(hist_pred_class[0]) -1
                prob = hist_pred_class[-1]
                probs_sums[class_number] += prob * (time_stamp+1) #rank, timestamp could be 0
        self.casting_hist.append(probs_sums)
        visualize_time_list(self.casting_hist[-len(hist) : ], name = dataplot_name)

        #norm result to one answer string
        max_prob_class = np.argmax(probs_sums) +1 #returns the index of the maximum ; +1 to get the original class
        return str(max_prob_class)


    ## 5. arithmetic mean -> sum existence; weight in regard to rank position (1., 2., or 3. most likely prediction) *if difference is small, consider if you really make a decision or if you wait*    
    def casting5(self, hist, dataplot_name = 'casting5.png'):
        rank_list = np.zeros(29)
        for time_stamp in np.arange(len(hist)):    #possible prefactor timeline
            hist_pred = hist[time_stamp]
            for rank in np.arange(len(hist_pred)): #possible prefactor prediction rank
                hist_pred_class = hist_pred[rank]
                class_number = int(hist_pred_class[0]) -1
                rank_list[class_number] += 1/ (rank+1) #Consider: rank, timestamp could be 0 
        self.casting_hist.append(rank_list)
        visualize_time_list(self.casting_hist[-len(hist) : ], name = dataplot_name)

        #norm result to one answer string
        max_prob_class = np.argmax(rank_list) +1 #returns the index of the maximum ; +1 to get the original class
        return str(max_prob_class)    


    ## 6. arithmetic mean -> sum existence; weight in regard to rank position (1., 2., or 3. most likely prediction) *if difference is small, consider if you really make a decision or if you wait*    
    def casting6(self, hist, dataplot_name = 'casting6.png'):
        rank_list = np.zeros(29)
        for time_stamp in np.arange(len(hist)):    #possible prefactor timeline
            hist_pred = hist[time_stamp]
            for rank in np.arange(len(hist_pred)): #possible prefactor prediction rank
                hist_pred_class = hist_pred[rank]
                class_number = int(hist_pred_class[0]) -1
                rank_list[class_number] += 1/ (rank+1)**2 *(time_stamp+1) #Consider: rank, timestamp could be 0 
        self.casting_hist.append(rank_list)
        visualize_time_list(self.casting_hist[-len(hist) : ], name = dataplot_name)

        #norm result to one answer string
        max_prob_class = np.argmax(rank_list) + 1 #returns the index of the maximum ; +1 to get the original class
        return str(max_prob_class)    

    ## 7. arithmetic mean -> sum existence; weight in regard to rank position (1., 2., or 3. most likely prediction) *if difference is small, consider if you really make a decision or if you wait*    
    def casting7(self, hist, dataplot_name = 'casting7.png'):
        rank_list = np.zeros(29)
        for time_stamp in np.arange(len(hist)):    #possible prefactor timeline
            hist_pred = hist[time_stamp]
            for rank in np.arange(len(hist_pred)): #possible prefactor prediction rank
                hist_pred_class = hist_pred[rank]
                class_number = int(hist_pred_class[0]) -1
                prob = hist_pred_class[-1]
                rank_list[class_number] += prob / ( (len(hist)-time_stamp) )#Consider: rank, timestamp could be 0 
        self.casting_hist.append(rank_list)
        visualize_time_list(self.casting_hist[-len(hist) : ], name = dataplot_name)

        #norm result to one answer string
        max_prob_class = np.argmax(rank_list) + 1 #returns the index of the maximum ; +1 to get the original class
        return str(max_prob_class)    


        
        

In [26]:
def setup():
    global hist, propabilities_hist, Castings, display_surface, topk
    topk = 3 #evtl global? oder 
    
    ##the pygame window 
    pygame.init() 
    pygame.font.init()

    X = 1500
    Y = 800

    display_surface = pygame.display.set_mode((X, Y))
    
    pygame.display.set_caption('ASL') 
    
    hist = []
    propabilities_hist = []
    Castings = []
    for i, casting in enumerate(['2', '4', '6', '7']):
        name =['2: ', '4: ', '6: ', '7: '][i]
        sent_pos = [(50, 600), (50, 630), (50, 660), (50, 690)][i]
        plot_pos = [(0,0), (1100,270), (680,270), (1100,0)][i]
        dataplot_name = [r'casting2', r'casting4.png', r'casting6.png', r'casting7.png'][i]
        SearchInstance = DetectSentence(name, casting, sent_pos, plot_pos = plot_pos, dataplot_name = dataplot_name)
        Castings.append(SearchInstance)
        
def main_loop_v2(hist_length):       
    global hist, propabilities_hist, Castings, display_surface, topk
    
    myfont = pygame.font.SysFont('Comic Sans MS', 30) ##For changes: Copied to the DetectSentence - Class
    white = (255, 255, 255) 
    black = (0, 0, 0)

    for turn in np.arange(60):
        
        display_surface.fill(white) 
        
        takeImage()
        probs, classes = predict(r'test.png', model, topk = topk) 
        #append last prediction to hist ## Consider using np.array (and leaving the str (e.g. "A") out for data type conformity)        
        hist.append( [ [classes[i] , number_to_ASL[classes[i]] , probs[i] ] for i in np.arange(topk)] )
        
        for i in np.arange(3):
            pos = ((50, 500), (50, 530), (50, 560))[i]
            text = str(number_to_ASL[classes[i]]) + ': ' + str(probs[i])
            textsurface = myfont.render(text, False, black)
            display_surface.blit(textsurface, pos)       
        info  = 'Cycle N°: ' + str(turn) + ';  hist_length: ' + str(len(hist[-hist_length : ])) #+ '; topk: ' + str(topk)
        infosurface  = myfont.render(info , False, (0,0,255))
        display_surface.blit(infosurface, (700, 560))
        
        #if needed, crop hist to uniform length
        #call DetectSentence
        for casting in Castings:
            if len(hist)< hist_length: 
                #casting.draw_on_pygame(display_surface, hist)
                casting.refresh_sentence(hist)
            else:
                casting.refresh_sentence(hist[-hist_length : ])
            casting.draw_on_pygame(display_surface)    
        
        propabilities = np.zeros(29)
        for i in range(topk):
            propabilities[int(classes[i])-1] = probs[i]
        #visualize as 3D Plot of a timeline (Heatmap):
        propabilities_hist.append(propabilities)
        visualize_time_list(propabilities_hist[-hist_length : ], name = '3D-heatmap-probs over time.png')
        probs_plot = pygame.image.load(r'3D-heatmap-probs over time.png')
        display_surface.blit(probs_plot, (680, 0))
        
        image = pygame.image.load(r'test.png')
        display_surface.blit(image, (0, 0)) 

        #cropped_image = pygame.image.load(r'cropped.png')
        #display_surface.blit(cropped_image, (700,0))
        
        pygame.display.update() 

        for event in pygame.event.get() : 
            if event.type == pygame.QUIT or event.type == 'KEYDOWN' :
                pygame.quit() 

                quit() 
                cam.release()
                pass
            
        

In [27]:
setup()

In [28]:
import time
if __name__ == "__main__":
    cam = cv2.VideoCapture(0)
    takeImage()
    
    hist_length = 20 ### Maximum number of predictions which are taken into account in the computations
    main_loop_v2(hist_length)
    
    cam.release()
    #time.sleep(8)
    #pygame.quit()

#Teste Tastendruck -> Abbruchkriterium

import cv2

import time

while True:
    time.sleep(1)
    k = cv2.waitKey(0)
    print(k)
    if k == 27:
        break

In [29]:
print(Castings[-1].sentence)

7: AAAAAAAAAAAAAAAAABB
